In [1]:
"""Copyright © by Boston Consulting Group. All rights reserved."""
import datetime
import logging
import kaleido
import pathlib
import os
from typing import Any, Dict, List, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import ticker
from sklearn import metrics
from tqdm import tqdm
from sklearn.metrics import auc, roc_curve
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.graph_objects as go

pd.set_option('display.max_columns', None)

import plotly.io as pio
pio.renderers.default = "iframe"

In [2]:
base_path = pathlib.Path().resolve().parent
data_path = base_path / 'data'
sample_data_loc = data_path / 'test_sample.csv'
df_sample = pd.read_csv(sample_data_loc)

In [7]:
def create_binary_target_plots(
    df: pd.DataFrame,
    target_col: str,
    feature_col: str,
    n_bins: int = 10,
    min_feature_val: float = None,
    max_feature_val: float = None,
    min_feature_q: float = None,
    max_feature_q: float = None,
    create_plot=True,
    invert_xaxis=False,
    use_quantiles: bool = True,
    fillna: bool = False
) -> Tuple[Union[pd.DataFrame, None], Union[plt.figure, None]]:
    """Creates three plots for univariate analysis for binary targets.
    :param df: Input data frame
    :param target_col: Column name of target variable
    :param feature_col: List of column names contained in df.
    :param n_bins: Number of bins
    :param min_feature_val: Lower feature value at which to
        winsorize (only applied to numerical columns).
    :param max_feature_val Upper feature value at which to
        winsorize (only applied to numerical columns).
    :param min_feature_q: Lower quantile at which to
        winsorize (only applied to numerical columns).
    :param max_feature_q: Upper quantile at which to
        winsorize (only applied to numerical columns).
    :param create_plot: Determines whether figure will be plotted.
    :param use_quantiles: Whether to use quantiles for event rate bins.
    :return: -> Dataframe containing feature and target metrics and figure:
    """

    if df[feature_col].nunique() < 2:
        _logger.info("Feature contains less than 2 features!")
        return None, None

    # Ensure target contains no NA values
    if df[target_col].isna().sum() > 0:
        raise ValueError("Target column contains NA values!")

    # Ensure target is binary
    if df[target_col].nunique() != 2:
        raise ValueError("Target variable is not binary!")

    # Ensure min/max values are in correct order
    if (
        min_feature_val is not None
        and max_feature_val is not None
        and min_feature_val >= max_feature_val
    ):
        raise ValueError(
            "min_feature_value must be strictly smaller than max_feature_val!"
        )
    elif (
        min_feature_q is not None
        and max_feature_q is not None
        and min_feature_q >= max_feature_q
    ):
        raise ValueError("min_feature_q must be strictly smaller than max_feature_q!")
    elif min_feature_q is not None and min_feature_q < 0:
        raise ValueError("Quantiles must be values between 0 and 1!!")
    elif max_feature_q is not None and max_feature_q < 0:
        raise ValueError("Quantiles must be values between 0 and 1!!")

    # Create fresh copy of dataframe
    df = df.copy()
    
    # Calculate global event rate
    event_rate = np.mean(df[target_col])

    # Case 1: At most n_bins elements
    if df[feature_col].nunique() < n_bins:

        # Extract unique values
        unique_vals = df[feature_col].sort_values().unique()

        # Convert sorted elements to int
        mapping = pd.factorize(unique_vals, na_sentinel=len(unique_vals))

        # Create mapping
        mapping_values = list(mapping[0])
        mapping_keys = list(mapping[1])

        # Create mapping dictionary
        mapping_dict = dict(zip(mapping_keys, mapping_values))

        # Apply mapping to column
        df = df.assign(bins=df[feature_col].map(mapping_dict))

        # Set labels for plotting
        labels = mapping_keys

    # Case 2: More than n_bins elements and categorical
    elif (not pd.api.types.is_numeric_dtype(df[feature_col])) and (
        df[feature_col].nunique() >= n_bins
    ):
        _logger.info(
            f"--- Number of unique elements ({df[feature_col].nunique()} "
            f"exceeds n_bins ({n_bins})). "
            f"Increase n_bins to create a plot!"
        )
        return None, None

    # Case 3: More than n_bins elements and numerical
    elif pd.api.types.is_numeric_dtype(df[feature_col]):
        

        # Extract number of unique values
        n_unique_feat_vals = df[feature_col].nunique()
        print(f"n_unique_feat_vals: {n_unique_feat_vals}") # 48

        # Adjust n_bins is less unique values exist
        n_bins = np.minimum(n_unique_feat_vals, n_bins)
        print(f"n_bins: {n_bins}") # min(48, 10) = 10

        min_val = np.nanmin(df[feature_col])
        max_val = np.nanmax(df[feature_col])
        print(f"min: {min_val}, max: {max_val}")
        
        # override min and max if quantiles passed to f'n
        if min_feature_q is not None:
            min_val = np.nanquantile(df[feature_col], min_feature_q)

        if max_feature_q is not None:
            max_val = np.nanquantile(df[feature_col], max_feature_q)

        if min_feature_val is not None:
            min_val = np.maximum(min_val, min_feature_val)

        if max_feature_val is not None:
            max_val = np.minimum(max_val, max_feature_val)
        
        # get min & max excluding +/- inf
        idx_neg_inf = df[feature_col] == -np.inf
        idx_pos_inf = df[feature_col] == np.inf
        min_val_excl_inf = np.nanmin(df[feature_col][~idx_neg_inf])
        max_val_excl_inf = np.nanmax(df[feature_col][~idx_pos_inf])

        min_val_adj = np.maximum(min_val, min_val_excl_inf)
        max_val_adj = np.minimum(max_val, max_val_excl_inf)

        # Create equidistant grid
        if use_quantiles:
            print('using quantiles')
            
            # using unique ensures that when quantiles are the same they are not used twice
            bins = np.unique(
                df[feature_col]
                .clip(min_val_adj, max_val_adj)
                .quantile(np.linspace(0, 1, n_bins))
            )
            print(bins)
            n_bins = len(bins)
        else:
            print('not using quantiles')
            bins = np.linspace(min_val_adj, max_val_adj, n_bins)
    
        
        print(f"bins before updating: {bins}")
        bins[0] = min_val
        bins[n_bins - 1] = max_val
        print(f"bins after updating: {bins}")

        # Clip values according to min/max values
        df[feature_col].clip(lower=min_val, upper=max_val, inplace=True)

        # Ensure clipping values does not remove all but a single value
        if df[feature_col].nunique() < 2:
            _logger.info(
                "Feature contains less than 2 features after clipping outliers!!"
            )
            return None, None

        # Create bins (return None if binning is not successfull)
        try:
            
            # create new column indicating what bin record belongs to
            df = df.assign(
                bins=pd.cut(
                    x=df.loc[:, feature_col],
                    bins=bins,
                    include_lowest=True,
                    right=True,
                    labels=False,
                )
            )
        except Exception as e:
            _logger.warning(e)
            return None, None
        
        # Create plot labels: [(4, 6), (6, 10), ...]
        bins = list(bins)
        labels = [
            f'({"{:,.2f}".format(bins[i])}, {"{:,.2f}".format(bins[i+1])}]'
            for i in range(n_bins - 1)
        ]
        print(labels)

        # Handle NAs
        if df["bins"].isna().sum() > 0:
            
            # replace the NA bin w/ n_bins - 1
            df.loc[:, "bins"] = df.loc[:, "bins"].where(
                ~df.loc[:, "bins"].isna(), n_bins - 1
            )
            labels.append("NA")
            n_bins += 1

        # Convert bins to categories
        df.bins = df.bins.astype("category")

        # Set all categories
        df.bins = df.bins.cat.set_categories(list(range(n_bins - 1)))

    # Group into bins and calculate required metrics
    df_binned = df.groupby("bins").agg({feature_col: [len], target_col: ["mean"]})
    print(df_binned)

    # Rename columns
    level_one = df_binned.columns.get_level_values(0).astype(str)
    level_two = df_binned.columns.get_level_values(1).astype(str)
    column_separator = ["_" if x != "" else "" for x in level_two]
    print(level_one + column_separator + level_two)
    df_binned.columns = level_one + column_separator + level_two
    print(df_binned.columns)


    # Set NA counts to zero
    df_binned[f"{feature_col}_len"] = df_binned[f"{feature_col}_len"].fillna(0)

    # Add lift rate
    df_binned.loc[:, "lift_rate"] = np.divide(
        df_binned[f"{target_col}_mean"], event_rate
    )

    # Build plots dependent on feature type
    if not pd.api.types.is_numeric_dtype(df[feature_col]):

        pass

    else:
        
        ## NEW PLOTLY FUNCTION
        fig = make_subplots(
                rows=2, 
                cols=2,
                vertical_spacing=0.1,   # decrease the vertical spacing between rows (adjust as desired)
                horizontal_spacing=0.1,  # decrease the horizontal spacing between columns (adjust as desired)
                specs=[[{}, {}],
                       [{"colspan": 2, "secondary_y": True}, None]],
                )
        
        if fillna == True:
            df_imputed = df.copy()
            df_imputed.loc[df_imputed[feature_col].isin([-np.inf, np.inf, np.nan]), feature_col] = df_imputed.loc[~df_imputed[feature_col].isin([-np.inf, np.inf, np.nan]), feature_col].median()
            
        else:
            df_imputed = df.loc[~df[feature_col].isin([-np.inf, np.inf, np.nan])]
        fpr, tpr, _ = metrics.roc_curve(
            df_imputed[target_col], -df_imputed[feature_col], pos_label=1
        )
        auc = metrics.auc(fpr, tpr)
        if auc < 0.5:
            fpr, tpr, _ = metrics.roc_curve(
                df_imputed[target_col], df_imputed[feature_col], pos_label=1
            )
            auc = metrics.auc(fpr, tpr)

        # Build Roc-Curve plot
        if pd.api.types.is_numeric_dtype(df[feature_col]):
            
            ## PLOTLY PLOT 1
            print(fpr)
            fig.add_trace(
                go.Scatter(
                    x=fpr, 
                    y=tpr,
                    mode='lines',
                    line= dict(
                        width = 1,
                    ),
                    hoverinfo='skip',
                    showlegend=False,
                ), 
                row = 1, 
                col = 1)
            
            fig.add_trace(
                go.Scatter(
                    x=np.linspace(0, 1, 10), 
                    y=np.linspace(0, 1, 10),
                    mode='lines',
                    line= dict(
                        color = 'rgba(169,169,169,0.5)',
                        dash  = 'dash',
                        width = 0.5,
                    ),
                    hoverinfo='skip',
                    showlegend=False,
                ), 
                row = 1, 
                col = 1)
            
            fig.update_xaxes(
                title_text="Cumulated goods", 
                title_font = {"size": 12},
                range=[0, 1],
                row=1, 
                col=1,
                title_standoff = 5 #decrease space between title and plot
            )
            
            fig.update_yaxes(
                title_text="Cumulated bads", 
                title_font = {"size": 12},
                range=[0, 1.05],
                row=1, 
                col=1,
                title_standoff = 5 #decrease space between title and plot
            )
            
            fig.add_annotation(
                x=0.65,
                y=0.1,
                xref="x1",
                yref="y1",
                text=f"area = {auc:.3f}",
                showarrow=False,
                bordercolor='rgba(255,255,255,1)',
                borderwidth=2,
                borderpad=4,
                bgcolor='rgba(255,255,255,1)',
                opacity=0.8
                )
            
        #if all(df[feature_col] > 0):
        #    upper_ax_right.set_xscale("log")
            
            
        # PLOTLY distplot
        hist_data = [
            df_imputed.loc[(df_imputed[target_col]==0), feature_col].values, 
            df_imputed.loc[(df_imputed[target_col]==1), feature_col].values
        ]
        group_labels = ['0', '1']
        fig2 = ff.create_distplot(hist_data, group_labels)
        
        max_density = max(fig2['data'][2].y.max(), fig2['data'][3].y.max())
        
        fig.add_trace(
            go.Scatter(
                fig2['data'][2],
                line=dict(
                    color='blue', 
                    width=0.5
                ),
                fill='tonexty',
                fillcolor='rgba(0, 0, 255, 0.2)',
                hoverinfo='skip',
                
            ), 
            row=1, 
            col=2,
        )

        fig.add_trace(
            go.Scatter(
                fig2['data'][3],
                line = dict(
                    color='red', 
                    width=0.5),
                fill='tozeroy',
                fillcolor='rgba(255, 0, 0, 0.2)',
                hoverinfo='skip',
            ), 
            row=1, 
            col=2,
        )
        
        fig.update_yaxes(
                title_text="Density", 
                title_font = {"size": 12},
                row=1, 
                col=2,
                title_standoff = 5 #decrease space between title and plot
            )
        
        fig.add_annotation(
                x=0.9 * max_val_adj,
                y=1 * max_density,
                xref="x2",
                yref="y2",
                text=f"Class: 0",
                font=dict(
                    color='rgba(0, 0, 255, 1)'
                ),
                showarrow=False,
                bordercolor='rgba(255,255,255,1)',
                borderwidth=2,
                borderpad=4,
                bgcolor='rgba(255,255,255,1)',
                opacity=0.8
                )
        
        fig.add_annotation(
                x=0.9 * max_val_adj,
                y=0.9 * max_density,
                xref="x2",
                yref="y2",
                text=f"Class: 1",
                font=dict(
                    color='rgba(255, 0, 0, 1)'
                ),
                showarrow=False,
                bordercolor='rgba(255,255,255,1)',
                borderwidth=2,
                borderpad=4,
                bgcolor='rgba(255,255,255,1)',
                opacity=0.8
                )


    ## PLOTLY BOTTOM PLOT
    print(df_binned.index)
    fig.add_trace(
        go.Bar(
            x=labels, 
            y=df_binned[f"{feature_col}_len"],
            marker_color='rgba(0,0,255,0.1)', 
            marker_line_color='rgba(0,0,255,1)',
            marker_line_width=1.5, 
            opacity=0.6,
            showlegend=False,
            hoverinfo='skip',
            
        ), 
        row=2, 
        col=1, 
        secondary_y=False
    )
    
    fig.add_trace(
        go.Scatter(
            x=labels, 
            y=df_binned[f"{target_col}_mean"],
            mode='lines+markers',
            line= dict(
                color = 'rgba(255,0,0,1)',
                width = 1,
            ),
            hoverinfo='skip',
            name='Event Rate',
        ), 
        row=2, 
        col=1, 
        secondary_y=True)
    
    fig.add_trace(
        go.Scatter(
            x=labels, 
            y= [event_rate] * len(labels),
            mode='lines',
            line= dict(
                color = 'rgba(169,169,169,1)',
                dash = 'dash',
                width = 1,
            ),
            hoverinfo='skip',
            name=f"General Event Rate: ({'{:.1%}'.format(event_rate)})",
        ), 
        row=2, 
        col=1, 
        secondary_y=True)
    
    fig.update_xaxes(
                title_text=f"{feature_col}", 
                title_font = {"size": 12},
                row=2, 
                col=1,
                title_standoff = 5, #decrease space between title and plot
                tickangle=22.5
            )
    
    fig.update_yaxes(
                title_text=f"# Observations", 
                title_font = {"size": 12},
                row=2, 
                col=1,
                title_standoff = 5, #decrease space between title and plot
            )
    
    fig.layout.yaxis4.title.text = 'Event Rate'
    
    fig.update_layout(
            showlegend=True
           ,title = dict(
               text = f"{feature_col}: Roc Curve | Densities | Event Rates", 
               font= dict(
                   size=20
               )
           )
            ,margin=dict(t=40)
            ,title_x=0.5
            ,width=1000
            ,height=800
            ,legend=dict(
                 x=0.9,
                 y=0.36,
                 xanchor="right",
                 yanchor="bottom",
             )
        )
    
    if invert_xaxis:
        lower_ax_2.invert_xaxis()

    if create_plot:
        fig.show()
        
    if not os.path.exists("images"):
        os.mkdir("images")
    
    fig.write_image(f"images/univariate_analysis_{feature_col}.jpeg")

    return fig

In [8]:
fig = create_binary_target_plots(
    df = df_sample,
    target_col = "FLAG_60_DPD_366_DAYS",
    feature_col = "LN_LEXISNEXIS_SBFE_SCORE_CURRENT_N"
)

n_unique_feat_vals: 251
n_bins: 10
min: 501.0, max: 900.0
using quantiles
[501.         662.77777778 689.         707.33333333 719.
 732.         747.         770.         799.         900.        ]
bins before updating: [501.         662.77777778 689.         707.33333333 719.
 732.         747.         770.         799.         900.        ]
bins after updating: [501.         662.77777778 689.         707.33333333 719.
 732.         747.         770.         799.         900.        ]
['(501.00, 662.78]', '(662.78, 689.00]', '(689.00, 707.33]', '(707.33, 719.00]', '(719.00, 732.00]', '(732.00, 747.00]', '(747.00, 770.00]', '(770.00, 799.00]', '(799.00, 900.00]']
     LN_LEXISNEXIS_SBFE_SCORE_CURRENT_N FLAG_60_DPD_366_DAYS
                                    len                 mean
bins                                                        
0                                   100             0.040000
1                                   101             0.019802
2                     